## Feature engineering

In [3]:
import pandas as pd

In [8]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))

from feature_engineering.indicators import simple_moving_average, momentum, exponential_moving_average, bollinger_bands, macd

In [9]:
data = pd.read_csv('../data/binance/BTC/1h.csv')

In [10]:
# Apply indicators on data

# Apply Simple Moving Average (SMA)
data = simple_moving_average(data, 'Close', window=5)
data = simple_moving_average(data, 'Close', window=10)

# Apply Momentum (MTM)
data = momentum(data, 'Close', window=5)
data = momentum(data, 'Close', window=10)

# Apply Exponential Moving Average (EMA)
data = exponential_moving_average(data, 'Close', span=5)
data = exponential_moving_average(data, 'Close', span=10)

# Apply Bollinger Bands (BB)
data = bollinger_bands(data, 'Close', window=20)

# Apply Moving Average Convergence Divergence (MACD)
data = macd(data, 'Close')

In [11]:
data.head()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,...,EMA_5,EMA_10,MA,STD,UB,LB,EMA_short,EMA_long,MACD,Signal_Line
0,2024-09-17 19:00:00,60540.17,60696.48,59618.26,59967.99,2112.05320,2024-09-17 19:59:59.999,1.270612e+08,322417,881.13029,...,59967.990000,59967.990000,NaN,NaN,NaN,NaN,59967.990000,59967.990000,0.000000,0.000000
1,2024-09-17 20:00:00,59967.98,60342.85,59715.42,60120.00,1665.06151,2024-09-17 20:59:59.999,9.996920e+07,208296,828.74091,...,60018.660000,59995.628182,NaN,NaN,NaN,NaN,59991.376154,59979.250000,12.126154,2.425231
2,2024-09-17 21:00:00,60119.99,60424.25,60105.01,60334.07,662.68788,2024-09-17 21:59:59.999,3.997326e+07,85064,356.50218,...,60123.796667,60057.163058,NaN,NaN,NaN,NaN,60044.098284,60005.532963,38.565321,9.653249
3,2024-09-17 22:00:00,60334.07,60343.99,60122.50,60240.01,424.37655,2024-09-17 22:59:59.999,2.556103e+07,94677,173.20763,...,60162.534444,60090.407956,NaN,NaN,NaN,NaN,60074.238548,60022.901632,51.336916,17.989982
4,2024-09-17 23:00:00,60240.01,60324.00,60005.31,60313.99,489.52738,2024-09-17 23:59:59.999,2.945738e+07,73556,267.66050,...,60213.019630,60131.059237,NaN,NaN,NaN,NaN,60111.123387,60044.463734,66.659653,27.723916
